In [77]:
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
import numpy as np

class bossHandler:
    def __init__(self, collection_name):
        """
        Constructor
        """
        self.collection_name = collection_name
        try:
            self.rmt = BossRemote()
        except:
            print('Unexpected Error:', sys.exc_info()[0])
    
#     def get_collection_list(self):
#         return self.rmt.list_collections()

    def list_experiments(self):
        """
        List all the experiments available in current collection
        """
        exp_list = self.rmt.list_experiments(self.collection_name)
        return exp_list
    
    def select_experiment(self, experiment_name):
        """
        Select an experiment to be added to this handler
        """
        tmp = ExperimentResource(collection_name = self.collection_name, name = experiment_name)
        exp = self.rmt.get_project(tmp)
        self.experiment = exp
#         return exp
    
    def get_experiment(self):
        """
        Return the currently selected experiment for this handler
        """
        if hasattr(self,'experiment'):
            
            return self.experiment
        else:
            raise AttributeError('No experiment exists. First, select an experiment using select_experiment')
            
            
    def list_channels(self):
        """
        List all channel in currently selected experiment
        """
        return self.rmt.list_channels(self.collection_name, self.experiment.name)
        
    def select_channel(self, channel_name):
        
        """
        Select a channel to be added to this handler
        """
        self.channel = self.rmt.get_channel(chan_name= channel_name, coll_name=self.collection_name, exp_name=self.experiment.name)
    
    def get_coordinate_frame(self):
        """
        Get current experiment's coordinate frame
        """
        tmp = CoordinateFrameResource(name=self.experiment.coord_frame)
        coor = self.rmt.get_project(tmp)
        self.coordinate_frame = coor
        return coor
    
    def get_all(self):
        """
        Get a the entire channel image data at its native resolution
        """
        x_rng = [self.coordinate_frame.x_start , self.coordinate_frame.x_stop]
        y_rng = [self.coordinate_frame.y_start , self.coordinate_frame.y_stop]
        z_rng = [self.coordinate_frame.z_start , self.coordinate_frame.z_stop]
        
        return rmt.get_cutout(self.channel , 0 , x_rng , y_rng , z_rng)
        
    def get_cutout(self, x_range, y_range, z_range, resolution=0):
        """
        Return a cutout of the image data
        """
        return self.rmt.get_cutout(self.channel , resolution , x_range , y_range , z_range)
        
        
        
        
    

In [48]:
bHandler = bossHandler('ailey-dev')
# bHandler.list_experiments()
bHandler.select_experiment('Insula_Atenolol-1_171204_new')
exp = bHandler.get_experiment()
bHandler.list_channels()
bHandler.select_channel('Ch0')
coor = bHandler.get_coordinate_frame()

In [ ]:
from skimage import filters



x_rng = [300 , 500]
y_rng = [700 , 900]
z_rng = [450 , 600]

data_cutout = bHandler.get_cutout(x_rng , y_rng , z_rng)

In [73]:
val = filters.threshold_otsu(data_cutout)
print(val)

6345


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

x_rng = [300 , 500]
y_rng = [700 , 900]
z_rng = [450 , 600]

data_cutout = bHandler.get_cutout(x_rng , y_rng , z_rng)

fig = plt.figure()
im = plt.imshow(data_cutout[0,:,:],animated=True)

def animate(i):
    im.set_data(data_cutout[i,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, frames = np.arange(0,data_cutout.shape[0]), interval = 100)
HTML(anim.to_jshtml())


In [ ]:
# print(np.min(data_cutout))
threshold = (np.max(data_cutout) - np.min(data_cutout))/3

data_cutout[data_cutout < threshold] = 0
data_cutout[data_cutout >= threshold] = 1

In [ ]:
print(threshold)

In [ ]:
fig = plt.figure()
im = plt.imshow(data_cutout[0,:,:],animated=True)

def animate(i):
    im.set_data(data_cutout[i,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, frames = np.arange(0,data_cutout.shape[0]), interval = 100)
HTML(anim.to_jshtml())

In [ ]:
print(np.max(data_cutout))

In [ ]:
from skimage import exposure as imex
data_cutout_normalized = np.zeros(data_cutout.shape)

for idx in np.arange(0,data_cutout.shape[0]):
    data_cutout_normalized[idx] = imex.equalize_hist(data_cutout[idx])


In [ ]:
fig = plt.figure()
im = plt.imshow(data_cutout_normalized[0,:,:],animated=True)

def animate(i):
    im.set_data(data_cutout_normalized[i,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, frames = np.arange(0,data_cutout_normalized.shape[0]), interval = 100)
HTML(anim.to_jshtml())



In [ ]:
# Thresholding for binarization

threshold = 0.85
from skimage import exposure as imex
data_cutout_normalized = np.zeros(data_cutout.shape)

for idx in np.arange(0,data_cutout.shape[0]):
    data_cutout_normalized[idx] = imex.equalize_hist(data_cutout[idx])


data_cutout_normalized[data_cutout_normalized < threshold] = 0
data_cutout_normalized[data_cutout_normalized >= threshold] = 1

fig = plt.figure()
im = plt.imshow(data_cutout_normalized[0,:,:],animated=True)

def animate(i):
    im.set_data(data_cutout_normalized[i,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, frames = np.arange(0,data_cutout_normalized.shape[0]), interval = 100)
HTML(anim.to_jshtml())